Procesamiento de datos del segundo fomulario. El que hice en Exactas y Fadu con aprox 15 personas en casa uno. 

Los datos originales estan aca:

https://github.com/IoniUTDT/Magia/tree/master/PilotoFacuFadu

El proyecto Magia es un proyecto que busca medir como responde la gente frente a una situacion novedosa y dificil de explicar en terminos fisicos. La idea es hacer un "truco" (el del buzo en una botella de agua) donde no es obvio porque el globo sube o baja, disfrazado de algo "sorprendente" y pedirle a la gente que complete una encuesta sobre que cree que esta pasando. Buscamos ver cuanto inventa la gente y con que sesgo en funcion de su formacion profesional. 

Los datos de la encuesta son levantados automaticamente mediante el sotware sdaps que genera los formularios y despues mediante escaneo los procesa en bloque sin necesidad de pasar los datos a manos a una compu. Del programa se puede exportar un csv con los datos en bruto. Donde a cada entrada le corresponde una encuesta completada. 

In [1]:
%load_ext autoreload
%autoreload 2
import pandas
from IPython.display import display

En el siguiente bloque cargamos los datos y los procesamos para que sean mas legibles. Para eso hay que interpretar el csv donde viene guardados. El [csv](./DataValidada_PilotoKonex.csv) se genero automaticamente por el spdaps a partir de escanear el [archivo](./questionnaire.pdf), donde como se puede ver, hay secciones, preguntas y opciones multiples. Cada seccion esta indicada en el encabezado por un primer numero, cada preguntas por un segundo numero y cada opcion por un tercero unidos con un _ luego cada fila representa una entrada donde un 1 representa seleccionado en el caso de las opciones multiples o el valor seleccionado en caso de que se tratara de una escala. 

In [4]:
data = pandas.read_csv('./Data_Exp2.csv')
with open('Codigos.txt') as f:
    lineas = f.readlines()

grupos = []
subGrupo = []

for linea in lineas:
    if linea == "\n":
        grupos.append(subGrupo)
        subGrupo = []
    else:
        subGrupo.append(linea.split('\n')[0])
if not subGrupo == []:
    grupos.append(subGrupo)

for grupo in grupos:
    encabezado = grupo[0]
    multiples = False
    if encabezado[-2:] == '_M':
        display ('Eha!')
        display (encabezado)
        multiples=True
        encabezado = encabezado[:-2]
    inicio = grupo[1]
    opciones = grupo[2::]
    
    if len(opciones)>0:
        titulos = [inicio+'_'+str(indice) for indice in range(len(opciones))]
    else:
        titulos = [grupo[1]]
    mapeo = {x:y for x,y in zip(titulos,opciones)}
    
    if len(titulos) > 1:
        display (multiples)
        if not multiples:
            display ('Eha!NoMu')
            display (encabezado)
            data[encabezado] = data[titulos].idxmax(axis=1)
            data[encabezado].replace(mapeo, inplace=True)
            data.loc[data[titulos].sum(axis=1)<1, encabezado] = 'NS/NC'
            data.loc[data[titulos].sum(axis=1)>1, encabezado] = 'Null'
        else:
            display ('Eha!SiMu')
            display (encabezado)
 
            data[encabezado] = [[]] * len(data)
            
            for titulo in titulos:
                print (mapeo[titulo])
                data[encabezado] = [x if y == 0 else x + [mapeo[titulo]] for x,y in zip(data[encabezado],data[titulo])]
    else:
        data[encabezado] = data[titulos]
    data = data.drop(titulos, 1)

'Eha!'

'Edad_M'

True

'Eha!SiMu'

'Edad'

<5
5-12
13-17
18-24
25-40
>40


In [5]:
# data = pandas.read_csv('./Data_Exp2.csv')
display (data.head())

,questionnaire_id,global_id,1_2_0,1_2_1,1_2_2,1_2_3,1_2_4,1_2_5,1_2_6,1_2_7,...,4_1_17_1,4_1_17_2,4_1_17_3,4_1_17_4,4_1_18_0,4_1_18_1,4_1_18_2,4_1_18_3,4_1_18_4,Edad
0,['31256'],1002,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,[18-24]
1,['59406'],1002,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,1,0,0,[18-24]
2,['55654'],1002,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,[18-24]
3,['15766'],1002,0,1,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,[25-40]
4,['61935'],1002,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,[18-24]
